In [ ]:
import os
import gc
import joblib
import traceback
import warnings
import numpy as np
import pandas as pd
import polars as pl
import tensorflow as tf
from pathlib import Path
from scipy.stats import skew, kurtosis, SmallSampleWarning
from scipy.spatial.transform import Rotation as R
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow import shape, minimum
from tensorflow.keras import backend as k
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import pad_sequences, Sequence, to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Dense, Input, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Concatenate,
    BatchNormalization, GRU, Dropout, add, Activation, Multiply, Reshape,
    LayerNormalization, Add, Bidirectional, LSTM, UpSampling1D, Lambda, GaussianNoise
)

In [ ]:
IS_KAGGLE = os.getenv('KAGGLE_KERNEL_RUN_TYPE') is not None

if IS_KAGGLE:
    print("Running on Kaggle environment.")
    # Example paths for Kaggle - YOU WILL NEED TO ADJUST THESE
    # Path to your pre-computed feature dataset
    INPUT_DIR = Path("/kaggle/input/your-feature-dataset-name") 
    # Path where artifacts (models, scalers) will be written
    OUTPUT_DIR = Path("/kaggle/working/")
    # Path to the original raw competition data
    RAW_DATA_DIR = Path("/kaggle/input/cmi-detect-behavior-with-sensor-data")
else:
    print("Running on local environment.")
    # Adjust these paths to match your local project structure
    INPUT_DIR = Path("output") 
    OUTPUT_DIR = Path("output")
    RAW_DATA_DIR = Path("input/cmi-detect-behavior-with-sensor-data")

# --- File & Directory Paths ---
PARQUET_FILE = INPUT_DIR / 'final_model_input_dataset.parquet' # Example name
PRETRAINED_DIR = OUTPUT_DIR / "artifacts"
PRETRAINED_DIR.mkdir(parents=True, exist_ok=True)

# --- Model & Training Hyperparameters ---
LR_INIT = 5e-4
WD = 3e-3
NUM_CLASSES = 18
BATCH_SIZE = 64
N_SPLITS = 4 
MAX_PAD_LEN = 128
RANDOM_STATE = 42

In [ ]:
def se_block(x, reduction=8):
    ch = x.shape[-1]
    se = GlobalAveragePooling1D()(x)
    se = Dense(ch // reduction, activation='relu')(se)
    se = Dense(ch, activation='sigmoid')(se)
    se = Reshape((1, ch))(se)
    return Multiply()([x, se])

def residual_se_cnn_block(x, filters, kernel_size, pool_size=2, drop=0.3, wd=1e-4):
    shortcut = x
    for _ in range(2):
        x = Conv1D(filters, kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
    x = se_block(x)
    if shortcut.shape[-1] != filters:
        shortcut = Conv1D(filters, 1, padding='same', use_bias=False, kernel_regularizer=l2(wd))(shortcut)
        shortcut = BatchNormalization()(shortcut)
    x = add([x, shortcut])
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size)(x)
    x = Dropout(drop)(x)
    return x

def crop_or_pad(inputs):
    x, skip = inputs
    x_len = shape(x)[1]
    skip_len = shape(skip)[1]
    min_len = minimum(x_len, skip_len)
    return x[:, :min_len, :], skip[:, :min_len, :]

def crop_or_pad_output_shape(input_shapes):
    shape1, shape2 = input_shapes
    min_time_steps = min(shape1[1], shape2[1]) if shape1[1] is not None and shape2[1] is not None else None
    num_features = shape1[2]
    output_shape = (None, min_time_steps, num_features)
    return [output_shape, output_shape]

def match_time_steps(x, skip):    
    x, skip = Lambda(crop_or_pad, output_shape=crop_or_pad_output_shape)([x, skip])
    return x, skip

def res_se_cnn_decoder_block(x, filters, kernel_size, drop=0.3, wd=1e-4, skip_connection=None):
    x = UpSampling1D(size=2)(x)
    x = Conv1D(filters, kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x)
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    if skip_connection is not None:
        x, skip_connection = match_time_steps(x, skip_connection)
        x = Concatenate()([x, skip_connection])
    x = Conv1D(filters, kernel_size, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x)
    x = LayerNormalization()(x)
    x = Activation('relu')(x)
    x = se_block(x)
    x = Dropout(drop)(x)
    return x

def unet_se_cnn(x, unet_depth=3, base_filters=64, kernel_size=3, drop=0.3):
    filters = base_filters
    skips = []
    for _ in range(unet_depth):
        x = residual_se_cnn_block(x, filters, kernel_size, drop=drop)
        skips.append(x)
        filters *= 2
    c_shape = x.shape[-1]
    x = Dense(128)(x)
    x = Dense(c_shape)(x)
    for skip in reversed(skips):
        filters //= 2
        x = res_se_cnn_decoder_block(x, filters, kernel_size, drop=drop, skip_connection=skip)
    return x

def tof_block(tof_inputs, wd=1e-4):
    x2_base = Conv1D(64, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(tof_inputs)
    x2_base = BatchNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    x2_base = MaxPooling1D(2)(x2_base); x2_base = Dropout(0.2)(x2_base)
    x2_base = Conv1D(128, 3, padding='same', use_bias=False, kernel_regularizer=l2(wd))(x2_base)
    x2_base = BatchNormalization()(x2_base); x2_base = Activation('relu')(x2_base)
    gate_input = GlobalAveragePooling1D()(tof_inputs)
    gate_input = Dense(16, activation='relu')(gate_input)
    gate = Dense(1, activation='sigmoid', name='tof_gate_dense')(gate_input)
    return Multiply()([x2_base, gate])

def time_sum(x): return k.sum(x, axis=1)
def squeeze_last_axis(x): return tf.squeeze(x, axis=-1)
def expand_last_axis(x): return tf.expand_dims(x, axis=-1)

def attention_layer(inputs):
    score = Dense(1, activation='tanh')(inputs)
    score = Lambda(squeeze_last_axis)(score)
    weights = Activation('softmax')(score)
    weights = Lambda(expand_last_axis)(weights)
    context = Multiply()([inputs, weights])
    context = Lambda(time_sum)(context)
    return context    

def features_processing(x1, x2, wd=1e-4):
    x1_matched, x2_matched = match_time_steps(x1, x2)
    merged = Concatenate()([x1_matched, x2_matched])
    xa = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xb = Bidirectional(GRU(128, return_sequences=True, kernel_regularizer=l2(wd)))(merged)
    xc = GaussianNoise(0.09)(merged)
    xc = Dense(16, activation='elu')(xc)
    x = Concatenate()([xa, xb, xc])
    x = Dropout(0.4)(x)
    x = attention_layer(x)
    for units, drop in [(256, 0.5), (128, 0.3)]:
        x = Dense(units, use_bias=False, kernel_regularizer=l2(wd))(x)
        x = BatchNormalization()(x); x = Activation('relu')(x)
        x = Dropout(drop)(x)
    return x

# =====================================================================================
# SEGMENT 4: GENERAL HELPER FUNCTIONS
# =====================================================================================

class GatedMixupGenerator(Sequence):
    def __init__(self, X, y, gate_targets, batch_size, imu_dim, class_weight=None, alpha=0.2, masking_prob=0.0):
        self.X, self.y = X, y
        self.gate_targets = gate_targets  
        self.batch = batch_size
        self.imu_dim = imu_dim
        self.class_weight = class_weight
        self.alpha = alpha
        self.masking_prob = masking_prob
        self.indices = np.arange(len(X))
        self.on_epoch_end()
    def __len__(self): return int(np.ceil(len(self.X) / self.batch))
    def __getitem__(self, i):
        idx = self.indices[i*self.batch:(i+1)*self.batch]
        Xb, yb = self.X[idx].copy(), self.y[idx].copy()
        gate_target = self.gate_targets[idx].copy()
        if self.masking_prob > 0:
            for sample_idx in range(len(Xb)):
                if gate_target[sample_idx] == 1.0 and np.random.rand() < self.masking_prob:
                    Xb[sample_idx, :, self.imu_dim:] = 0
                    gate_target[sample_idx] = 0.0
        sample_weights = np.ones(len(Xb), dtype='float32')
        if self.class_weight:
            y_integers = yb.argmax(axis=1)
            sample_weights = np.array([self.class_weight[i] for i in y_integers])
        if self.alpha > 0:
            lam = np.random.beta(self.alpha, self.alpha)
            perm = np.random.permutation(len(Xb))
            X_mix = lam * Xb + (1 - lam) * Xb[perm]
            y_mix = lam * yb + (1 - lam) * yb[perm]
            gate_target_mix = lam * gate_target + (1 - lam) * gate_target[perm]
            sample_weights_mix = lam * sample_weights + (1 - lam) * sample_weights[perm]
            return X_mix, {'main_output': y_mix, 'tof_gate': gate_target_mix[:, np.newaxis]}, sample_weights_mix
        return Xb, {'main_output': yb, 'tof_gate': gate_target[:, np.newaxis]}, sample_weights    
    def on_epoch_end(self): np.random.shuffle(self.indices)

def create_sequence_dataset(df: pl.DataFrame, feature_cols: list, gate_df: pl.DataFrame):
    sequences, labels, gate_targets = [], [], []
    df_with_gate = df.join(gate_df, on='sequence_id', how='left')
    for seq_id, group in df_with_gate.group_by('sequence_id', maintain_order=True):
        sequences.append(group.select(feature_cols).to_numpy())
        labels.append(group.select('gesture_int').item(0, 0))
        gate_targets.append(group.select('has_tof').item(0, 0))
    return np.array(sequences, dtype=object), np.array(labels), np.array(gate_targets)

def train_model(model, train_dataset, val_dataset, epochs, initial_learning_rate, weight_decay):
    optimizer = tf.keras.optimizers.AdamW(learning_rate=initial_learning_rate, weight_decay=weight_decay)
    early_stopping = EarlyStopping(monitor='val_main_output_accuracy', patience=20, restore_best_weights=True, mode='max')
    model.compile(optimizer=optimizer, loss={'main_output': 'categorical_crossentropy', 'tof_gate': 'binary_crossentropy'},
                  loss_weights={'main_output': 1.0, 'tof_gate': 0.5}, metrics={"main_output": "accuracy"})
    model.fit(train_dataset, validation_data=val_dataset, epochs=epochs, callbacks=[early_stopping])

def generate_gate_targets(df: pl.DataFrame, tof_cols: list) -> pl.DataFrame:
    gate_df = df.group_by("sequence_id").agg(
        pl.any_horizontal(pl.col(tof_cols).is_not_null().any()).alias("has_tof")
    )
    return gate_df.with_columns(pl.col("has_tof").cast(pl.Float32))    

def perform_padding(X, pad_len):
    return pad_sequences(
        X,
        maxlen=pad_len,
        padding='post', # Pad at the end of the sequence
        dtype='float32',# Data type of the output tensor
        truncating='post',
        value=0.0,      # Value to use for padding (e.g., 0 for numerical data)
    )

In [ ]:
def remove_gravity_polars(acc_df: pl.DataFrame, rot_df: pl.DataFrame) -> np.ndarray:
    acc_values = acc_df.select(['acc_x', 'acc_y', 'acc_z']).to_numpy()
    quat_values = rot_df.select(['rot_x', 'rot_y', 'rot_z', 'rot_w']).to_numpy()
    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81])
    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            linear_accel[i] = acc_values[i]
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            gravity_sensor_frame = rotation.apply(gravity_world, inverse=True)
            linear_accel[i] = acc_values[i] - gravity_sensor_frame
        except ValueError:
            linear_accel[i] = acc_values[i]
    return linear_accel

def calculate_angular_velocity(rot_df: pl.DataFrame, sampling_rate_hz: int) -> np.ndarray:
    quats = rot_df.select(['rot_x', 'rot_y', 'rot_z', 'rot_w']).to_numpy()
    angular_velocity = np.zeros_like(quats[:, :3])
    dt = 1.0 / sampling_rate_hz
    for i in range(1, len(quats)):
        try:
            q1 = R.from_quat(quats[i - 1])
            q2 = R.from_quat(quats[i])
            q_delta = q2 * q1.inv()
            rot_vec = q_delta.as_rotvec()
            angular_velocity[i] = rot_vec / dt
        except ValueError:
            angular_velocity[i] = 0
    return angular_velocity

def calculate_angular_acceleration(angular_velocity: np.ndarray, sampling_rate_hz: int) -> np.ndarray:
    angular_accel = np.zeros_like(angular_velocity)
    dt = 1.0 / sampling_rate_hz
    angular_accel[1:] = np.diff(angular_velocity, axis=0) / dt
    return angular_accel

def calculate_gravity_orientation(rot_df: pl.DataFrame) -> np.ndarray:
    quat_values = rot_df.select(['rot_x', 'rot_y', 'rot_z', 'rot_w']).to_numpy()
    num_samples = quat_values.shape[0]
    orientation_angles = np.zeros((num_samples, 3))
    gravity_world = np.array([0, 0, 1.0])
    for i in range(num_samples):
        if np.all(np.isnan(quat_values[i])) or np.all(np.isclose(quat_values[i], 0)):
            continue
        try:
            rotation = R.from_quat(quat_values[i])
            sensor_axes_world = rotation.apply(np.eye(3))
            for j in range(3):
                dot_product = np.dot(sensor_axes_world[j], gravity_world)
                orientation_angles[i, j] = np.arccos(np.clip(dot_product, -1.0, 1.0))
        except ValueError:
            continue
    return orientation_angles

def pl_skew(s: pl.Series) -> float:
    values = s.to_numpy()
    if len(values) < 3: return None
    if np.std(values) < 1e-9: return 0.0
    return skew(values)

def pl_kurtosis(s: pl.Series) -> float:
    values = s.to_numpy()
    if len(values) < 4: return None
    if np.std(values) < 1e-9: return 0.0
    return kurtosis(values)

def process_tof_features(df: pl.DataFrame) -> pl.DataFrame:
    # (The full Polars-based ToF feature generation function goes here)
    # ...
    return df # Placeholder

In [ ]:
# =====================================================================================
# MASTER CONTROL FLAG
# =====================================================================================
TRAIN = False
TRAIN = True 

# =====================================================================================
# MODEL DEFINITION 
# =====================================================================================
def create_model(dataset, imu_dim, wd=1e-4):
    sample_batch = next(iter(dataset))
    input_shape = sample_batch[0].shape[1:]
    inp = tf.keras.layers.Input(shape=input_shape)
    imu = tf.keras.layers.Lambda(lambda t: t[:, :, :imu_dim])(inp)
    tof = tf.keras.layers.Lambda(lambda t: t[:, :, imu_dim:])(inp)

    x1 = unet_se_cnn(imu, 3, base_filters=64, kernel_size=3)
    x2 = tof_block(tof, wd)

    x = features_processing(x1, x2)
    x = tf.keras.layers.Dropout(0.3)(x) 
    main_out = tf.keras.layers.Dense(18, activation="softmax", name="main_output")(x)
    gate_out = tf.keras.layers.Dense(1, activation="sigmoid", name="tof_gate")(x) 
    
    return tf.keras.models.Model(inputs=inp, outputs={"main_output": main_out, "tof_gate": gate_out})

# =====================================================================================
# TRAINING LOGIC
# =====================================================================================
if TRAIN:
    schema_df = pl.read_parquet(PARQUET_FILE, n_rows=0)
    all_columns = schema_df.columns
    meta_cols = {'gesture', 'gesture_int', 'sequence_type', 'behavior', 'orientation',
                    'row_id', 'subject', 'phase', 'sequence_id', 'sequence_counter'}
    feature_cols = [c for c in all_columns if c not in meta_cols]
    imu_cols  = [c for c in feature_cols if not (c.startswith('thm_') or c.startswith('tof_'))]
    tof_cols  = [c for c in feature_cols if c.startswith('thm_') or c.startswith('tof_')]

    print("Scanning Parquet file for sequence IDs...")
    all_sequence_ids = (
        pl.scan_parquet(PARQUET_FILE)
        .select('sequence_id')
        .unique()
        .collect()
        .to_numpy()
        .ravel()
    )
    print(f"Found {len(all_sequence_ids)} unique sequences.")

    kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
    fold_accuracies = []
    all_preds = []
    all_labels = []
    imu_dim = len(imu_cols)

    for fold_idx, (train_indices, val_indices) in enumerate(kf.split(all_sequence_ids)):
        print(f"\n=== Fold {fold_idx + 1}/{N_SPLITS} ===")
        train_ids = all_sequence_ids[train_indices]
        val_ids = all_sequence_ids[val_indices]

        print(f"Loading data for fold {fold_idx + 1}...")
        train_df = pl.read_parquet(PARQUET_FILE).filter(pl.col('sequence_id').is_in(train_ids))
        val_df = pl.read_parquet(PARQUET_FILE).filter(pl.col('sequence_id').is_in(val_ids))
        print("Fold data loaded.")

        train_gate_df = generate_gate_targets(train_df, tof_cols)
        val_gate_df = generate_gate_targets(val_df, tof_cols)

        le = LabelEncoder()
        le.fit(train_df['gesture'])
        train_df = train_df.with_columns(pl.Series("gesture_int", le.transform(train_df['gesture'])))
        val_df = val_df.with_columns(pl.Series("gesture_int", le.transform(val_df['gesture'])))

        # --- StandardScaler Logic ---
        scaler = StandardScaler()
        # Fit on training data and transform both
        train_features_scaled = scaler.fit_transform(train_df[imu_cols + tof_cols])
        val_features_scaled = scaler.transform(val_df[imu_cols + tof_cols])
        # Create Polars DataFrames from the scaled numpy arrays
        X_train_scaled_features = pl.DataFrame(train_features_scaled, schema=imu_cols + tof_cols)
        X_val_scaled_features = pl.DataFrame(val_features_scaled, schema=imu_cols + tof_cols)

        meta_cols_to_keep = ['sequence_id', 'gesture_int']
        train_df_final = train_df.select(meta_cols_to_keep).with_columns(X_train_scaled_features)
        val_df_final = val_df.select(meta_cols_to_keep).with_columns(X_val_scaled_features)

        del train_df, val_df, X_train_scaled_features, X_val_scaled_features
        gc.collect()

        X_train, y_train, train_gate_target = create_sequence_dataset(train_df_final, imu_cols + tof_cols, train_gate_df)
        X_val, y_val, val_gate_target = create_sequence_dataset(val_df_final, imu_cols + tof_cols, val_gate_df)

        del train_df_final, val_df_final
        gc.collect()

        X_train_padded = perform_padding(X_train, MAX_PAD_LEN)
        X_val_padded = perform_padding(X_val, MAX_PAD_LEN)
        
        y_train_cat = to_categorical(y_train, num_classes=NUM_CLASSES)
        y_val_cat = to_categorical(y_val, num_classes=NUM_CLASSES)

        train_dataset = GatedMixupGenerator(
            X=X_train_padded, y=y_train_cat, gate_targets=train_gate_target,
            batch_size=BATCH_SIZE, imu_dim=imu_dim, alpha=0.2, masking_prob=0.25
        )
        val_dataset = tf.data.Dataset.from_tensor_slices((
            X_val_padded, {'main_output': y_val_cat, 'tof_gate': val_gate_target[:, np.newaxis]}
        )).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

        del X_val, y_val, X_train, y_train, X_train_padded, X_val_padded
        gc.collect()
        
        model = create_model(train_dataset, len(imu_cols))
        train_model(model, train_dataset, val_dataset, 150, LR_INIT, WD)

        # --- SAVE ARTIFACTS ---
        print(f"--- Saving artifacts for Fold {fold_idx + 1} ---")
        model.save(PRETRAINED_DIR / f"gesture_model_fold_{fold_idx}.h5")
        
        # Save scaler and other metadata only from the first fold
        if fold_idx == 0:
            joblib.dump(scaler, PRETRAINED_DIR / "scaler.pkl")
            np.save(PRETRAINED_DIR / "feature_cols.npy", np.array(imu_cols + tof_cols))
            np.save(PRETRAINED_DIR / "sequence_maxlen.npy", MAX_PAD_LEN)
            np.save(PRETRAINED_DIR / "gesture_classes.npy", le.classes_)
            print("Scaler, feature_cols, maxlen, and classes saved.")

        # --- EVALUATION ---
        val_preds = model.predict(val_dataset)
        main_output_preds = val_preds['main_output']
        y_pred_fold = np.argmax(main_output_preds, axis=1)
        y_true_fold = np.argmax(y_val_cat, axis=1)
        fold_acc = accuracy_score(y_true_fold, y_pred_fold)
        fold_accuracies.append(fold_acc)
        print(f"Fold {fold_idx + 1} Accuracy: {fold_acc:.4f}")
        all_preds.append(y_pred_fold)
        all_labels.append(y_true_fold)

        del train_dataset, model, val_dataset
        gc.collect()

    # --- FINAL OOF REPORT ---
    print("\n=== Cross-validation Summary ===")
    print(f"Per-fold Accuracies: {fold_accuracies}")
    print(f"Mean Accuracy: {np.mean(fold_accuracies):.4f} ± {np.std(fold_accuracies):.4f}")
    y_all_pred = np.concatenate(all_preds)
    y_all_true = np.concatenate(all_labels)
    print("\n=== Overall Classification Report ===")
    print(classification_report(y_all_true, y_all_pred, target_names=le.classes_, digits=4))

# =====================================================================================
# INFERENCE LOGIC
# =====================================================================================
else:
    import pandas as pd 
    from src.metric import CompetitionMetric 
    from tensorflow import argmax, minimum, shape

    def crop_or_pad(inputs):
        x, skip = inputs
        x_len = shape(x)[1]
        skip_len = shape(skip)[1]
        min_len = minimum(x_len, skip_len)
        return x[:, :min_len, :], skip[:, :min_len, :]
    
    # --- 1. Load All Inference Artifacts ---
    print("▶ INFERENCE MODE – loading artefacts from", PRETRAINED_DIR)
    final_feature_cols = np.load(PRETRAINED_DIR / "feature_cols.npy", allow_pickle=True).tolist()
    pad_len = int(np.load(PRETRAINED_DIR / "sequence_maxlen.npy"))
    scaler = joblib.load(PRETRAINED_DIR / "scaler.pkl")
    gesture_classes = np.load(PRETRAINED_DIR / "gesture_classes.npy", allow_pickle=True)

    print("Scaler expects:", list(scaler.feature_names_in_))
    print("We have:", final_feature_cols)
    print("Missing:", set(scaler.feature_names_in_) - set(final_feature_cols))
    print("Extra:", set(final_feature_cols) - set(scaler.feature_names_in_))

    models = []
    print(f"  Loading {N_SPLITS} models for ensemble inference...")
    for fold in range(N_SPLITS):
        model_path = PRETRAINED_DIR / f"gesture_model_fold_{fold}.h5"
        
        model = load_model(model_path, compile=False, custom_objects={
            'unet_se_cnn': unet_se_cnn,
            'tof_block': tof_block,
            'features_processing': features_processing,
            'match_time_steps': match_time_steps,
            'crop_or_pad': crop_or_pad,
            'squeeze_last_axis': squeeze_last_axis,
            'expand_last_axis': expand_last_axis,
            'time_sum': time_sum,
            'crop_or_pad_output_shape': crop_or_pad_output_shape
        })
        models.append(model)
    print("  Models, scaler, and metadata loaded – ready for evaluation.")

    # --- 2. Define TTA Parameters and Predict Function ---
    TTA_STEPS = 10
    TTA_NOISE_STDDEV = 0.01

    from src.tof_feats import remove_gravity_from_acc, calculate_angular_velocity_from_quat, calculate_angular_distance

    def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
        # Convert to pandas for the processing pipeline
        df_seq = sequence.to_pandas()

        # =================================================================================
        # --- Step 1: Sanitize Raw Inputs ---
        # =================================================================================
        # This is a robust guard against non-numeric data in the hidden test set.
        sensor_cols = [c for c in df_seq.columns if c.startswith(('acc_', 'rot_', 'thm_', 'tof_'))]
        for col in sensor_cols:
            if df_seq[col].dtype == 'object':
                df_seq[col] = pd.to_numeric(df_seq[col], errors='coerce')

        # =================================================================================
        # --- Step 2: Feature Engineering (Must match training pipeline exactly) ---
        # =================================================================================
        # Create features in a dictionary to avoid fragmenting the DataFrame.
        new_features = {}

        # --- IMU Features ---
        linear_accel = remove_gravity_from_acc(df_seq, df_seq)
        new_features['linear_acc_x'] = linear_accel[:, 0]
        new_features['linear_acc_y'] = linear_accel[:, 1]
        new_features['linear_acc_z'] = linear_accel[:, 2]
        
        linear_acc_mag = np.sqrt(np.square(linear_accel).sum(axis=1))
        new_features['linear_acc_mag'] = linear_acc_mag
        new_features['linear_acc_mag_jerk'] = pd.Series(linear_acc_mag).diff().fillna(0).values
        
        angular_vel = calculate_angular_velocity_from_quat(df_seq)
        new_features['angular_vel_x'] = angular_vel[:, 0]
        new_features['angular_vel_y'] = angular_vel[:, 1]
        new_features['angular_vel_z'] = angular_vel[:, 2]
        
        new_features['angular_distance'] = calculate_angular_distance(df_seq)

        # --- ToF Aggregated Features ---
        # This is the part that was missing.
        for i in range(1, 6):
            pixel_cols = [f"tof_{i}_v{p}" for p in range(64)]
            # The underlying tof_v... columns are already sanitized.
            # We still replace -1 as it's a specific code, not bad data.
            tof_data = df_seq[pixel_cols].replace(-1, np.nan) 
            new_features[f'tof_{i}_mean'] = tof_data.mean(axis=1)
            new_features[f'tof_{i}_std'] = tof_data.std(axis=1)
            new_features[f'tof_{i}_min'] = tof_data.min(axis=1)
            new_features[f'tof_{i}_max'] = tof_data.max(axis=1)

        # Add all new features to the DataFrame in one efficient operation
        df_seq = df_seq.assign(**new_features)

        # =================================================================================
        # --- Step 3: Final Processing (Scaling and Padding) ---
        # =================================================================================

        mat_unscaled_df = df_seq[final_feature_cols]
        mat_unscaled_filled = mat_unscaled_df.ffill().bfill().fillna(0)
        mat_scaled = scaler.transform(mat_unscaled_filled)
        pad_input = pad_sequences([mat_scaled], maxlen=pad_len, padding='post', truncating='post', dtype='float32')

        # =================================================================================
        # --- Step 4: TTA and Ensemble Prediction ---
        # =================================================================================
        all_tta_predictions = []
        for i in range(TTA_STEPS):
            noisy_input = pad_input
            if i > 0:
                noise = tf.random.normal(shape=tf.shape(pad_input), mean=0.0, stddev=TTA_NOISE_STDDEV)
                noisy_input = pad_input + noise

            all_fold_predictions = []
            for model in models:
                # Correctly unpack the dictionary returned by the model
                predictions_dict = model.predict(noisy_input, verbose=0)
                main_preds = predictions_dict['main_output']
                all_fold_predictions.append(main_preds)
            
            avg_fold_prediction = np.mean(all_fold_predictions, axis=0)
            all_tta_predictions.append(avg_fold_prediction)

        # =================================================================================
        # --- Step 5: Final Averaging and Prediction ---
        # =================================================================================
        final_avg_prediction = np.mean(all_tta_predictions, axis=0)
        idx = int(final_avg_prediction.argmax())
        
        return str(gesture_classes[idx])
    
    # --- 3. Run Kaggle Evaluation Server ---
    import kaggle_evaluation.cmi_inference_server
    inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

    if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
        inference_server.serve()
    else:
        # For local testing, you need to provide the paths to the test data
        print("Running local gateway for testing...")
        inference_server.run_local_gateway(
            data_paths=(
                'input/cmi-detect-behavior-with-sensor-data/test.csv',
                'input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
            )
        )